In [2]:
%load_ext Cython


The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [1]:
%reload_ext Cython


In [1]:
import cairo

VENDOR_ID = 0x046d
PRODUCT_ID = 0xc21c
INTERFACE = 0
MODE_LED_CONTROL = 0x301 # Could be 0x302?
COLOR_CONTROL = 0x301 # Could be 0x307?
KEY_ENDPOINT = 1
REPORT_SIZE = 8
#REQUEST_TYPE = libusb1.LIBUSB_TYPE_CLASS | libusb1.libusb_request_recipient.get('LIBUSB_RECIPIENT_INTERFACE')

LCD_WIDTH = 160  # X
LCD_HEIGHT = 44  # Y

lcd_surface = cairo.ImageSurface(cairo.FORMAT_RGB24, LCD_WIDTH, LCD_HEIGHT)
lcd_context = cairo.Context(lcd_surface)
lcd_context.set_source_rgb(1, 1, 1)
lcd_context.select_font_face('Verdana')
lcd_context.set_font_size(35)
lcd_surface_buffer = lcd_surface.get_data()

def reset(context, text):
    context.set_operator(cairo.OPERATOR_CLEAR)
    context.paint()
    context.show_text(text)

len(lcd_surface_buffer)



28160

In [5]:
reset(lcd_context, 'google')
lcd_surface.write_to_png('blahblah.png')



In [5]:
def set_color(handle, color, color_control, request_type):
    data = ''.join(map(chr, [7, color[0], color[1], color[2], 0]))
    handle.controlWrite(
        request_type=request_type, request=9,
        value=color_control, index=0, data=data.encode(),
        timeout=1000)

ddd 
b'\x07ddd\x00'


In [13]:
%%cython

cimport cython
import cairo
from cpython.array cimport array
from cpython.buffer cimport PyObject_GetBuffer

cdef inline _rgb_r(unsigned int pixel_value):
    #define RGB_R(x) ((x&0x00FF0000) >> 16)
    return pixel_value & 0x00FF0000 >> 16

cdef inline _rgb_g(unsigned int pixel_value):
    #define RGB_G(x) ((x&0x0000FF00) >> 8)
    return pixel_value & 0x0000FF00 >> 8

cdef inline _rgb_b(unsigned int pixel_value):
    #define RGB_B(x) ((x&0x000000FF) >> 0)
    return pixel_value & 0x000000FF >> 0


In [ ]:
%%cython

cimport cython
import cairo
from cpython.array cimport array
from cpython.buffer cimport PyObject_GetBuffer

cdef write_disp_matrix(unsigned int width, unsigned int threshold, array surface_buffer):
    # calling semantics:
    # self.surface = cairo.ImageSurface(cairo.FORMAT_RGB24, g13.LCD_WIDTH, g13.LCD_HEIGHT)
    # self.context = cairo.Context(self.surface)
    # surface_buffer = cairo.surface.get_data()

    cdef unsigned source_len = len(surface_buffer)
    cdef Py_buffer* destination_buffer

    cdef unsigned row = 0, col = 0
    cdef size_t max_iterations
    cdef char threshold_byte = threshold

    cdef int pixel_index = 0
    cdef int pixel_value = 0
    cdef bint show_pixel = False
    cdef int lcd_idx = 0








In [13]:
import usb1

def open_device():
    VENDOR_ID = 0x046d
    PRODUCT_ID = 0xc21c
    INTERFACE = 0
    MODE_LED_CONTROL = 0x301 # Could be 0x302?
    COLOR_CONTROL = 0x301 # Could be 0x307?
    KEY_ENDPOINT = 1
    REPORT_SIZE = 8
    #REQUEST_TYPE = libusb1.LIBUSB_TYPE_CLASS | libusb1.libusb_request_recipient.get('LIBUSB_RECIPIENT_INTERFACE')

    LCD_WIDTH = 160  # X
    LCD_HEIGHT = 44  # Y
    
    ctx = usb1.USBContext()
    dev = ctx.getByVendorIDAndProductID(VENDOR_ID, PRODUCT_ID)

    handle = dev.open()
    #if platform.system() == 'Linux' and \
        #handle.kernelDriverActive(INTERFACE):
    if handle.kernelDriverActive(INTERFACE):
            handle.detachKernelDriver(INTERFACE)

    handle.claimInterface(INTERFACE)
    return ctx, handle


In [3]:
def close_device(context, handle):
    handle.releaseInterface(INTERFACE)
    handle.close()
    context.exit()


In [14]:
request_mask = usb1.REQUEST_TYPE_CLASS | usb1.RECIPIENT_INTERFACE
device = open_device()

33